# SEIRX model runs for test technology and screening frequency investigation in schools

In [3]:
import networkx as nx
import pandas as pd
from os.path import join

import sys
sys.path.insert(0,'../school')
sys.path.insert(0,'../nursing_home')
from model_school import SEIRX_school
import analysis_functions as af

# for progress bars
from ipywidgets import IntProgress
from IPython.display import display
import time

**Note:** The number of simulation runs per scenario is set via the ```runs``` variable below. Running 10k simulations per scenario takes approximately 12 hours on a single ~4GHz core. Therefore, if you just want to test the simulation, I advise to set ```runs``` to for example 100. This will result in noisier statistics, but the trends will remain the same. 

In [7]:
# number of runs per scenario
runs = 50
# this is where simulation results will be stored
res_path = '../data/school' 
# file name of the results, will be appended by the number of runs
sim_name = 'school_results' 
# number of employees in the simulation. Note: the number of residents
# is automatically extracted from the contact network
N_employees = 18
# resident contact network
G = nx.readwrite.gpickle.read_gpickle('../data/school/test_volksschule.gpickle')
# maximum number of steps in a single run. A run automatically stops
# if the outbreak is contained, i.e. there are no more infected or
# exposed individuals
N_steps = 500 
# verbosity level of the simulation. Advised to be set to zero if 
# simulations are run for data creation purposes.
verbosity = 0
# student and teacher streening intervals (in days)
s_screen_range = [3, 7, 14, None]
t_screen_range = [3, 7, 14, None]
testing=True
# test technologies (and test result turnover times) used in the
# different scenarios
test_types = ['same_day_antigen', 'same_day_LAMP', 'same_day_PCR','one_day_PCR', 'two_day_PCR']
# specifies, whether the index case will be introduced via an
# employee or a resident
index_cases = ['student', 'teacher']
# agent types used in the simulation with their respective parameters
agent_types = {
        'student':{
            'screening_interval': None,
            'index_probability': 0,
            'transmission_risk': 0.0275,
            'reception_risk': 1,
            'symptom_probability': 0.6},
    
        'teacher':{
            'screening_interval': None,
            'index_probability': 0,
            'transmission_risk': 0.0275,
            'reception_risk': 1,
            'symptom_probability': 0.6},
    
        'family_member':{
            'screening_interval': None,
            'index_probability': 0,
            'transmission_risk': 0.0275,
            'reception_risk': 1,
            'symptom_probability': 0.6}
}

# progress bar
f = IntProgress(min=0, max=runs * len(s_screen_range) \
                * len(t_screen_range) * len(test_types) * len(index_cases)) 
display(f)
c=0 # counter for progress bar

scan_results = pd.DataFrame()

for ttype in test_types:
    for index_case in index_cases:
        for s_screen_interval in s_screen_range:
            agent_types['student']['screening_interval'] = s_screen_interval
            for t_screen_interval in t_screen_range:
                agent_types['teacher']['screening_interval'] = t_screen_interval
                # results of one ensemble, i.e. results of all runs
                # with the same parameters
                ensemble_results = pd.DataFrame()
                for r in range(runs):
                    f.value = c # update the progress bar
                    c += 1
                    
                    # instantiate model with current scenario settings
                    # instantiate model with current scenario settings
                    model = SEIRX_school(G, verbosity, testing=testing,
                      diagnostic_test_type = 'two_day_PCR',
                      preventive_screening_test_type = ttype,
                      index_case = index_case,
                      follow_up_testing_interval = None,
                      agent_types = agent_types)
                    
                    # run the model, end run if the outbreak is over
                    for i in range(N_steps):
                        model.step()
                        if len([a for a in model.schedule.agents if \
                            (a.exposed == True or a.infectious == True)]) == 0:
                            break
    
                    # collect the statistics of the single run
                    R0, _ = af.calculate_finite_size_R0(model)
                    infected_students = af.count_infected(model, 'student')
                    infected_teachers = af.count_infected(model, 'teacher')
                    infected_family_members = af.count_infected(model, 'family_member')
                    data = model.datacollector.get_model_vars_dataframe()
                    N_diagnostic_tests = data['N_diagnostic_tests'].max()
                    N_preventive_screening_tests = data['N_preventive_screening_tests'].max()
                    transmissions = sum([a.transmissions for a in model.schedule.agents])
                    infected_without_transmissions = af.count_infection_endpoints(model)
                    student_student_transmissions = af.count_typed_transmissions(model, 'student', 'student')
                    teacher_student_transmissions = af.count_typed_transmissions(model, 'teacher', 'student')
                    student_teacher_transmissions = af.count_typed_transmissions(model, 'student', 'teacher')
                    teacher_teacher_transmissions = af.count_typed_transmissions(model, 'teacher', 'teacher')
                    student_family_member_transmissions = af.count_typed_transmissions(model, 'student', 'family_member')
                    family_member_family_member_transmissions = af.count_typed_transmissions(model, 'family_member', 'family_member')
                    quarantine_days_student = model.quarantine_counters['student']
                    quarantine_days_teacher = model.quarantine_counters['teacher']
                    quarantine_days_family_member = model.quarantine_counters['family_member']
                    pending_test_infections = data['pending_test_infections'].max()
                    undetected_infections = data['undetected_infections'].max()
                    predetected_infections = data['predetected_infections'].max()
                    duration = len(data)
                    
                    # add run results to the ensemble results
                    ensemble_results = ensemble_results.append({'run':r, 
                      'R0':R0,
                      'infected_students':infected_students,
                      'infected_teachers':infected_teachers,
                      'infected_family_members':infected_family_members,
                      'N_diagnostic_tests':N_diagnostic_tests,
                      'N_preventive_tests':N_preventive_screening_tests,
                      'transmissions':transmissions,
                      'infected_without_transmissions':infected_without_transmissions,
                      'student_student_transmissions':student_student_transmissions,
                      'teacher_student_transmissions':teacher_student_transmissions,
                      'student_teacher_transmissions':student_teacher_transmissions,
                      'teacher_teacher_transmissions':teacher_teacher_transmissions,
                      'student_family_member_transmissions':student_family_member_transmissions,
                      'family_member_family_member_transmissions':family_member_family_member_transmissions,
                      'quarantine_days_student':quarantine_days_student,
                      'quarantine_days_teacher':quarantine_days_teacher,
                      'quarantine_days_family_member':quarantine_days_family_member,
                      'pending_test_infections':pending_test_infections,
                      'undetected_infections':undetected_infections,
                      'predetected_infections':predetected_infections,
                      'duration':duration},
                    ignore_index=True)
                    
               # add ensemble statistics to the overall results
                row = {'test_type':ttype,
                       'index_case':index_case,
                       'student_screen_interval':s_screen_interval,
                       'teacher_screen_interval':t_screen_interval}

                row.update(af.get_statistics(ensemble_results, 'R0'))
                row.update(af.get_statistics(ensemble_results, 'infected_students'))
                row.update(af.get_statistics(ensemble_results, 'infected_teachers'))
                row.update(af.get_statistics(ensemble_results, 'infected_family_members'))
                row.update(af.get_statistics(ensemble_results, 'N_diagnostic_tests'))
                row.update(af.get_statistics(ensemble_results, 'N_preventive_tests'))
                row.update(af.get_statistics(ensemble_results, 'transmissions'))
                row.update(af.get_statistics(ensemble_results, 'infected_without_transmissions'))
                row.update(af.get_statistics(ensemble_results, 'student_student_transmissions'))
                row.update(af.get_statistics(ensemble_results, 'teacher_student_transmissions'))
                row.update(af.get_statistics(ensemble_results, 'student_teacher_transmissions'))
                row.update(af.get_statistics(ensemble_results, 'teacher_teacher_transmissions'))
                row.update(af.get_statistics(ensemble_results, 'student_family_member_transmissions'))
                row.update(af.get_statistics(ensemble_results, 'family_member_family_member_transmissions'))
                row.update(af.get_statistics(ensemble_results, 'quarantine_days_student'))
                row.update(af.get_statistics(ensemble_results, 'quarantine_days_teacher'))
                row.update(af.get_statistics(ensemble_results, 'quarantine_days_family_member'))
                row.update(af.get_statistics(ensemble_results, 'pending_test_infections'))
                row.update(af.get_statistics(ensemble_results, 'undetected_infections'))
                row.update(af.get_statistics(ensemble_results, 'predetected_infections'))
                row.update(af.get_statistics(ensemble_results, 'duration'))

                scan_results = scan_results.append(row, ignore_index=True)

# save results to disk
scan_results.to_csv(join(res_path,'{}_N{}.csv'.format(sim_name, runs)), index=False)

IntProgress(value=0, max=8000)

In [8]:
scan_results

,N_diagnostic_tests_0.025,N_diagnostic_tests_0.25,N_diagnostic_tests_0.75,N_diagnostic_tests_0.975,N_diagnostic_tests_mean,N_diagnostic_tests_median,N_diagnostic_tests_std,N_preventive_tests_0.025,N_preventive_tests_0.25,N_preventive_tests_0.75,...,transmissions_mean,transmissions_median,transmissions_std,undetected_infections_0.025,undetected_infections_0.25,undetected_infections_0.75,undetected_infections_0.975,undetected_infections_mean,undetected_infections_median,undetected_infections_std
0,802.000,802.00,4797.00,21129.575,4636.46,1203.5,6486.305034,439.000,658.25,1094.00,...,14.40,1.0,28.599272,0.0,0.0,1.00,14.325,1.94,0.0,3.872509
1,802.000,802.00,3787.50,25558.900,4804.30,1603.0,7354.872743,420.000,619.00,1036.75,...,17.90,1.0,37.237928,0.0,0.0,1.00,8.325,1.34,0.0,2.544382
2,802.000,802.25,6182.25,24709.375,6063.46,1604.0,8022.966761,419.225,617.00,1134.50,...,24.98,1.0,44.778133,0.0,0.0,2.75,9.775,2.08,0.0,3.457261
3,802.000,803.00,16845.50,23572.100,7095.32,1604.0,8674.892863,399.000,598.25,1145.50,...,30.32,1.5,45.954476,0.0,0.0,2.00,13.550,1.98,0.0,3.678343
4,802.000,803.00,11862.25,20882.975,6051.86,1604.5,7289.364861,239.225,260.00,496.25,...,23.84,2.0,38.001804,0.0,0.0,1.75,3.775,0.90,0.0,1.297564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,3266.800,12416.75,19774.75,27103.125,15926.08,15726.5,6024.897106,0.000,0.00,0.00,...,91.78,104.5,38.347120,0.0,0.0,0.00,0.000,0.00,0.0,0.000000
156,1663.000,11988.25,19105.75,24202.700,14549.70,15911.0,6208.655799,0.000,0.00,0.00,...,76.74,89.5,41.499353,0.0,0.0,0.00,0.000,0.00,0.0,0.000000
157,3231.575,11780.00,19637.25,25505.950,15393.38,15870.5,6451.151923,0.000,0.00,0.00,...,81.46,96.0,39.985309,0.0,0.0,0.00,0.000,0.00,0.0,0.000000
158,2162.200,12880.50,18868.75,25346.450,14878.22,15432.0,5804.711195,0.000,0.00,0.00,...,83.78,98.0,40.440074,0.0,0.0,0.00,0.000,0.00,0.0,0.000000
